INTAKE

Purpose: The intake is like the ”entry door” for sunlight. It’s designed to gather as much sunlight as possible and
guide it into the Quantum Photonic Array.

Basic Idea: Imagine using a magnifying glass to focus sunlight on a tiny spot; the intake works on a similar
principle. It collects a large amount of sunlight and concentrates or ”focuses” it onto a smaller area, increasing the
energy densit

In [9]:
#   SUNLIGHT INTENSITY
#   The intensity of sunlight (I0(t)) on a clear day outside the Earth’s atmosphere is approximately 1361 W/m2
#   This intensity varies based on the angle of the sun and can be modeled using the cosine of the zenith angle:

#   Where day_of_year = 1 represents January 1st.


In [10]:
import math
import pandas as pd
intensity_at_top_of_atmosphere = 1361  # W/m^2
concentration_factor = 100

def solar_declination(d, h):
    # Convert angles to radians for calculations
    fracyear = 2*math.pi/365 * (d - 1 +(h - 12)/24)

    delta = math.degrees(0.006918 - 0.399912 * math.cos(fracyear) + 0.070257 * math.sin(fracyear) - 0.006758 * math.cos(2 * fracyear) + 0.000907 * math.sin(2 * fracyear) - 0.002697 * math.cos(3 * fracyear) + 0.00148 * math.sin(3 * fracyear))
    return delta

def calc_elevation_angle(d, ho, longitude=97.74, latitude=30.266666666666666):
    h = ho + 6
    fracyear = 2*math.pi/365 * (d - 1 +(h - 12)/24)
    # Equation of Time
    eqtime = 229.17 * (0.000075 + 0.001868 * math.cos(fracyear) - 0.032077 * math.sin(fracyear) - 0.014615 * math.cos(2 * fracyear) - 0.040849 * math.sin( 2 * fracyear))
    
    # Calculate the Time Offset
    time_offset = eqtime - 4 * longitude
    
    # True Solar Time in minutes
    tst = (h * 60 + time_offset) % 1440  # Modulo 1440 to keep within 24 hours
    
    # Hour angle
    if tst < 720:  # Before solar noon
        ha = math.radians(tst/4 - 180)
    else:  # After solar noon
        ha = math.radians(tst/4 - 180 - 360)
    
    delta = math.radians(solar_declination(d, h))
    
    zenith = math.degrees(math.acos(math.sin(math.radians(latitude)) * math.sin(delta) + math.cos(math.radians(latitude)) * math.cos(delta) * math.cos(ha)))

    psi = 90 - zenith
    return psi


def sunlight_intensity(solar_elevation_angle):
    return intensity_at_top_of_atmosphere * math.cos(math.radians(90 - solar_elevation_angle))


results = []
for day in range(0, 7, 1):  # Simulate each day

    for hour in range(0, 24, 1):  # Simulate each hour

        declination_angle = solar_declination(day, hour)
        elevation_angle = calc_elevation_angle(day, hour)
        intensity = sunlight_intensity(elevation_angle)
        concentrated_value = concentration_factor * intensity
        

        results.append([day, hour, declination_angle, elevation_angle, intensity, concentration_factor, concentrated_value])

df = pd.DataFrame(results, columns=["Simulated Day", "Hour of Day", "Solar Declination Angle", "Solar Elevation Angle (degrees)", "Sunlight Intensity (W/m^2)", "Concentration Factor", "Intensity * Concentration factor (funnel)"])
df.to_csv("solar_simulation_results.csv", index=False)